In [ ]:
# ==============================================================================
# CELL 1: BOOTSTRAP - INTELLIGENT ENVIRONMENT SETUP
# ==============================================================================
# This cell checks if the required dependencies are already installed. If not,
# it downloads and runs a setup script to prepare the Colab environment.

import sys
import subprocess
import os
import shutil

# --- Configuration ---
# Set to True to install the full OpenStudio CLI (required for running Measures)
INSTALL_CLI = False
# Set the branch of the toolkit to use ('main' for stable, 'dev' for development)
TOOLKIT_BRANCH = 'main'

# --- Verification and Execution Logic ---
try:
    # Attempt to import the core libraries.
    import openstudio
    import openstudio_toolkit
    
    # If INSTALL_CLI is True, also check for the command line interface.
    if INSTALL_CLI and not shutil.which('openstudio'):
        # This will trigger the installation if the CLI is missing but the library is present.
        raise ImportError("OpenStudio CLI not found")

    print("✅ Environment is already set up. Skipping installation.")

except ImportError:
    print("⏳ Required dependencies not found. Proceeding with setup...")
    
    bootstrap_script = 'bootstrap.py'
    # Download the bootstrap script if it doesn't exist OR if it's an empty file.
    if not os.path.exists(bootstrap_script) or os.path.getsize(bootstrap_script) == 0:
        print("--> Downloading bootstrap script...")
        bootstrap_url = f"https://raw.githubusercontent.com/roruizf/OpenStudio-Toolkit/{TOOLKIT_BRANCH}/{bootstrap_script}"
        os.system(f"wget -q -O {bootstrap_script} {bootstrap_url}")

    # Final check to ensure the downloaded script is not empty before running
    if os.path.exists(bootstrap_script) and os.path.getsize(bootstrap_script) > 0:
        print("--> Running environment setup...")
        cli_flag = "--install-cli" if INSTALL_CLI else ""
        os.system(f"python {bootstrap_script} --branch {TOOLKIT_BRANCH} {cli_flag}")
        
        print("\n💥 KERNEL RESTART is likely required for the changes to take effect.")
        print("Please go to 'Runtime' > 'Restart session' in the menu, then run the next cell.")
    else:
        print("❌ ERROR: Failed to download the bootstrap script. Please check the URL and your connection.")

In [ ]:
# ==============================================================================
# CELL 2: RUN TASK
# ==============================================================================
# IMPORTANT: Run this cell AFTER restarting the kernel from Cell 1 if needed.

import os
import pandas as pd

# --- Imports from the Toolkit ---
from openstudio_toolkit.utils.notebook_helpers import get_osm_path
from openstudio_toolkit.utils.osm_utils import load_osm_file_as_model, save_model_as_osm_file
from openstudio_toolkit.utils.excel_utils import read_excel_table_to_df
from openstudio_toolkit.tasks.model_setup import set_space_surfacearea_height_volume
from openstudio_toolkit.osm_objects.spaces import get_all_space_objects_as_dataframe

# --- User Inputs for File Access ---
USE_GDRIVE = False
GDRIVE_OSM_PATH = '/content/drive/MyDrive/path/to/your/model.osm'
GDRIVE_EXCEL_PATH = '/content/drive/MyDrive/path/to/your/data.xlsx'
LOCAL_OSM_PATH = '../../examples/Cabana-60.osm' 
LOCAL_EXCEL_PATH = '../../examples/Cabana-60-Data.xlsx' 

# --- Get File Paths ---
try:
    osm_file_path, project_folder = get_osm_path(
        use_gdrive=USE_GDRIVE, gdrive_path=GDRIVE_OSM_PATH, local_path=LOCAL_OSM_PATH)
    excel_file_path = GDRIVE_EXCEL_PATH if USE_GDRIVE and 'google.colab' in sys.modules else os.path.abspath(LOCAL_EXCEL_PATH)
except (ValueError, FileNotFoundError) as e:
    print(e)
    osm_file_path, excel_file_path = None, None

# --- Main Execution Block ---
if osm_file_path and excel_file_path and os.path.exists(osm_file_path) and os.path.exists(excel_file_path):
    
    file_name = os.path.basename(osm_file_path)
    output_file_name = f"{os.path.splitext(file_name)[0]}_geometry_updated.osm"
    output_osm_path = os.path.join(os.getcwd(), output_file_name)
    print(f"INFO: Output will be saved to: {output_osm_path}")

    # 1. Load Model and Prepare Data
    osm_model = load_osm_file_as_model(osm_file_path)
    space_data_df = read_excel_table_to_df(excel_file_path, sheet_name='Spaces', column_range='A:G')
    space_data_list = space_data_df.to_dict('records')
    
    # 2. Validate Model and Data
    print("\n--- Validating Model for Task ---")
    report = set_space_surfacearea_height_volume.validator(osm_model, spaces_data=space_data_list)
    for msg in report['messages']: print(msg)
    print("---------------------------------")
    
    # 3. Run Task and Save
    if report['status'] == 'READY':
        modified_model = set_space_surfacearea_height_volume.run(osm_model, spaces_data=space_data_list)
        save_model_as_osm_file(modified_model, output_osm_path)
        print(f"\n✅ Model saved successfully.")

        # 4. Verification Step
        print("\n--- Verifying Results ---")
        verified_model = load_osm_file_as_model(output_osm_path)
        final_spaces_df = get_all_space_objects_as_dataframe(verified_model)
        
        print("Displaying data for the first 5 spaces from the modified model:")
        display(final_spaces_df[['Name', 'Floor Area {m2}', 'Volume {m3}', 'Ceiling Height {m}']].head())
else:
    if not osm_file_path or not os.path.exists(osm_file_path):
        print(f"❌ ERROR: The specified OSM file path does not exist or was not provided.")
    if not excel_file_path or not os.path.exists(excel_file_path):
        print(f"❌ ERROR: The specified Excel file path does not exist or was not provided.")